# **Lection #7**:
### **Theme**: Патерны 2, Структурные 😭😲🤯
##### **Date**: 28.10.24

### **Паттерн** `Адаптер` 
- Преобразует интерфейс одного класса в интерфейс *другого*, который  
ожидают клиенты. Адаптер обеспечивает **совместную работу классов** с  
несовместимыми интерфейсами, которая без него была бы  
невозможна.  


In [2]:
class Target:
    def request(self) -> str:
        return "Target: the default target's behaviour"


class Adaptee:
    """
        я его адаптирую 🤯
    """
    def specific_request(self):
        return "Adaptee: special behavior"[::-1]


class Adapter(Target, Adaptee):
    def request(self) -> str:
        return f"Adapter: {self.specific_request()[::-1]}"


def client_code(target) -> None:
    print(target.request())


trgt = Target()
client_code(trgt)

# adptee = Adaptee()
adapter = Adapter()
client_code(adapter)

Target: the default target's behaviour
Adapter: Adaptee: special behavior


#### `Адаптер` через **композицию**


In [10]:
class AdapterComp(Target):
    def __init__(self, adaptee: Adaptee) -> None:
        self.adaptee = adaptee

    def request(self):
        return f"Adapter: {self.adaptee.specific_request()[::-1]}"

adaptee = Adaptee()
adapter2 = AdapterComp(adaptee)
client_code(adapter2)


Adapter: Adaptee: special behavior


### **Паттерн** `Фасад` 
- паттерн, *структурирующий* объекты.  
- **Фасад** определяет интерфейс более высокого уровня, который упрощает  
использование подсистемы.  


In [15]:
from __future__ import annotations

class Facade:
    def __init__(self, subsys1: Subsys1, subsys2: Subsys2) -> None:
        self._subsys1 = subsys1 or Subsys1()
        self._subsys2 = subsys2 or Subsys2()

    def operation(self) -> str:
        res = []
        res.append("Facade init subsys:")
        res.append(self._subsys1.operation1())
        res.append(self._subsys2.operation1())
        res.append("Facade orders subsys to perform the action:")
        res.append(self._subsys1.operation_n())
        res.append(self._subsys2.operation_z())
        return "\n".join(res)

class Subsys1:
    def operation1(str) -> str:
        return "Subsys 1 is ZZZZ!"

    def operation_n(str) -> str:
        return "Subsys 1 is CB0!"

class Subsys2:
    def operation1(str) -> str:
        return "Subsys 2 is VVVV!"

    def operation_z(str) -> str:
        return "Subsys 2 is ZOV!"

def client_code(facade: Facade):
    print(facade.operation())
    facade._subsys1.operation1()


sub1 = Subsys1()
sub2 = Subsys2()
facade = Facade(sub1, sub2)
client_code(facade)

Facade init subsys:
Subsys 1 is ZZZZ!
Subsys 2 is VVVV!
Facade orders subsys to perform the action:
Subsys 1 is CB0!
Subsys 2 is ZOV!


### **Паттерн** `Мост` 
- паттерн, *структурирующий* объекты.  
- отделить абстракцию от ее реализации так, чтобы то и другое  
можно было изменять независимо


In [18]:
from abc import ABC, abstractmethod

class Abstraction:
    def __init__(self, implementation: Implementation):
        self.implementation = implementation

    def operation(self):
        return f"Abstaction: Base operation with:\n{self.implementation.operation_imp()}"

class RefinedAbstraction(Abstraction):
    def operation(self):
        return f"Abstaction: Changed operation with:\n{self.implementation.operation_imp()}"


class Implementation(ABC):
    @abstractmethod
    def operation_imp(self):
        pass

class ConcreteImplementationA(Implementation):
    def operation_imp(self) -> str:
        return "ConcreteImplementation: result A"

class ConcreteImplementationB(Implementation):
    def operation_imp(self) -> str:
        return "ConcreteImplementation: result B"

def client_code(abstraction: Abstraction) -> None:
    print(abstraction.operation())

imp = ConcreteImplementationA()
abstraction = Abstraction(imp)
client_code(abstraction)

imp2 = ConcreteImplementationB()
abstraction2 = RefinedAbstraction(imp2)
client_code(abstraction2)


Abstaction: Base operation with:
ConcreteImplementation: result A
Abstaction: Changed operation with:
ConcreteImplementation: result B


# !DO NOT SUBMIT!, only test

In [19]:
import re

def tokenize(expression):
    if expression == "":
        return []

    regex = re.compile("\s*(=>|[-+*\/\%=\(\)]|[A-Za-z_][A-Za-z0-9_]*|[0-9]*\.?[0-9]+)\s*")
    tokens = regex.findall(expression)
    return [s for s in tokens if not s.isspace()]

class Interpreter:

    priority = ('%', '/', '*', '-', '+', '=')

    def __init__(self):
        self.vars = {}
        self.functions = {}

    def calc(self, a, b, op):
        if op == '=' and a.isalpha():
            self.vars[a] = b
            return float(b)
        if a in self.vars.keys(): a = self.vars[a]
        if b in self.vars.keys(): b = self.vars[b]

        if op == '+': return float(a) + float(b)
        elif op == '-': return float(a) - float(b)
        elif op == '*': return float(a) * float(b)
        elif op == '/' and b != '0': return float(a) / float(b)
        elif op == '%': return float(a) % float(b)

    def input(self, expression):
        tokens = tokenize(expression)
        if not tokens:
            return ''
        if (
                tokens.count('+') == 0
                and tokens.count('-') == 0
                and tokens.count('*') == 0
                and tokens.count('/') == 0
                and tokens.count('%') == 0
                and tokens.count('=') == 0
                and len(tokens) > 1
            ):
                raise Exception
        current_priority = 0
        while len(tokens) > 1:
            f = True
            for i in range(len(tokens)):
                if tokens[i] == ')':
                    j = i-1
                    res = []
                    while tokens[j] != '(':
                        res.append(tokens[j])
                        j -= 1
                    res = self.input(''.join(res[::-1]))
                    del tokens[j:i+1]
                    tokens.insert(j, str(res))
                    f = False
                    break

                if tokens.count(')') == 0 and tokens[i] in Interpreter.priority[current_priority]:
                    res = str(self.calc(tokens[i-1], tokens[i+1], tokens[i]))
                    print(tokens)
                    del tokens[i-1:i+2]
                    tokens.insert(i, res)
                    break

            if f:
                current_priority += 1
            if current_priority >= len(Interpreter.priority):
                break

        if tokens[0].isalpha() and not self.vars.get(tokens[0]):
            raise Exception
        if tokens[0].isalpha():
            return float(self.vars[tokens[0]])
        return float(tokens[0])


<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_32331/4096073864.py:7: SyntaxWarning: invalid escape sequence '\s'
  regex = re.compile("\s*(=>|[-+*\/\%=\(\)]|[A-Za-z_][A-Za-z0-9_]*|[0-9]*\.?[0-9]+)\s*")
